In [1]:
import numpy as np
import pandas as pd
import os 
import time
from selectolax.parser import HTMLParser
import requests
import datetime
import sqlite3

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### Extract the tables:

In [2]:
url = 'https://www.ssp.sp.gov.br/estatistica/ViolenciaMulher.aspx'

In [3]:
response = requests.get(url)
html = response.text

In [4]:
doc = HTMLParser(html)

In [5]:
tables = doc.css("table.table.table-striped.table-hover.table-condensed")

In [6]:
table_data = []

for table in tables:
    for row in table.css("tr"):
        row_data = []
        cells = row.css("th, td")  # Select both th and td elements
        for cell in cells:
            row_data.append(cell.text().strip())
        
        if any(row_data):
            table_data.append(row_data)

# Create a DataFrame from the extracted data
df = pd.DataFrame(table_data[1:], columns=table_data[0])

In [7]:
df.head(30)

,,Capital,Demacro,Interior,Total
0,HOMICÍDIO DOLOSO (exclui FEMINICÍDIO),2,2,7,11
1,FEMINICÍDIO,3,2,7,12
2,HOMICÍDIO DOLOSO - TOTAL,5,4,14,23
3,HOMICÍDIO CULPOSO,0,0,0,0
4,TENTATIVA DE HOMICÍDIO,12,5,19,36
5,LESÃO CORPORAL DOLOSA,1814,671,2164,4649
6,MAUS TRATOS,13,3,30,46
7,CALÚNIA - DIFAMAÇÃO - INJÚRIA,2454,559,2497,5510
8,CONSTRANGIMENTO ILEGAL,1,3,5,9
9,AMEAÇA,3087,931,3797,7815


### Extraindo as ocorrências:

In [8]:
url = 'https://www.ssp.sp.gov.br/estatistica/ViolenciaMulher.aspx'
response = requests.get(url)
html = response.text
doc = HTMLParser(html)
tables = doc.css("table.table.table-striped.table-hover.table-condensed")

In [9]:
# Create a list to store
ocorrencias = []

# Loop through table elements with IDs from conteudo_repPeriodo_lblPeriodo_2 and onward
table_number = 2  # Start from the second table
while True:
    table_id = f"conteudo_repPeriodo_lblPeriodo_{table_number}"
    span_element = doc.css(f'#{table_id}')
    
    # Check if the span element with the given ID exists
    if not span_element:
        break
    
    # Extract the text from the span element and append to the list
    ocorrencia = span_element[0].text()
    ocorrencias.append(ocorrencia)
    
    # Increment the table number for the next iteration
    table_number += 1

In [10]:
ocorrencias[0:4]

['Ocorrências Registradas no mês: Julho de 2023',
 'Ocorrências Registradas no mês: Junho de 2023',
 'Ocorrências Registradas no mês: Maio de 2023',
 'Ocorrências Registradas no mês: Abril de 2023']

In [11]:
print(len(tables), len(ocorrencias))

143 143


In [12]:
ocorrencia_time = pd.DataFrame(ocorrencias, columns=['ocorrencias'])
ocorrencias_br = pd.DataFrame(ocorrencias, columns=['ocorrencias'])

In [13]:
# Custom mapping of Portuguese to English month names
month_mapping = {
    'Janeiro': 'January',
    'Fevereiro': 'February',
    'Março': 'March',
    'Abril': 'April',
    'Maio': 'May',
    'Junho': 'June',
    'Julho': 'July',
    'Agosto': 'August',
    'Setembro': 'September',
    'Outubro': 'October',
    'Novembro': 'November',
    'Dezembro': 'December'
}

# Replace Portuguese month names with English month names
for pt_month, en_month in month_mapping.items():
    ocorrencia_time['ocorrencias'] = ocorrencia_time['ocorrencias'].str.replace(pt_month, en_month)
# Extract and convert the date
ocorrencia_time['date'] = ocorrencia_time['ocorrencias'].apply(lambda x: datetime.datetime.strptime(x.split(': ')[-1], '%B de %Y'))
ocorrencia_time = pd.concat([ocorrencia_time['date'],ocorrencias_br],axis=1)

In [14]:
ocorrencia_time

,date,ocorrencias
0,2023-07-01,Ocorrências Registradas no mês: Julho de 2023
1,2023-06-01,Ocorrências Registradas no mês: Junho de 2023
2,2023-05-01,Ocorrências Registradas no mês: Maio de 2023
3,2023-04-01,Ocorrências Registradas no mês: Abril de 2023
4,2023-03-01,Ocorrências Registradas no mês: Março de 2023
...,...,...
138,2012-01-01,Ocorrências Registradas no mês: Janeiro de 2012
139,2011-12-01,Ocorrências Registradas no mês: Dezembro de 2011
140,2011-11-01,Ocorrências Registradas no mês: Novembro de 2011
141,2011-10-01,Ocorrências Registradas no mês: Outubro de 2011


###  Criando o dataframe tabular:

In [15]:
col1 = []
for column in tables[0].css('tr')[1:]:
    column_data = str(column.css('td')[0].text().strip()).lower()
    col1.append(column_data)

In [16]:
col1

['homicídio doloso (exclui feminicídio)',
 'feminicídio',
 'homicídio doloso - total',
 'homicídio culposo',
 'tentativa de homicídio',
 'lesão corporal dolosa',
 'maus tratos',
 'calúnia - difamação - injúria',
 'constrangimento ilegal',
 'ameaça',
 'invasão de domicílio',
 'dano',
 'estupro consumado',
 'estupro tentado',
 'estupro de vulnerável consumado',
 'estupro de vulnerável tentado',
 'outros c/c/ dignidade sexual']

In [17]:
col2 = []
for column in tables[67].css('tr')[1:]:
    column_data = str(column.css('td')[0].text().strip()).lower()
    col2.append(column_data)

In [18]:
col2

['homicídio doloso (exclui feminicídio)',
 'homicídio culposo',
 'tentativa de homicídio',
 'lesão corporal dolosa',
 'maus tratos',
 'calúnia - difamação - injúria',
 'constrangimento ilegal',
 'ameaça',
 'invasão de domicílio',
 'dano',
 'estupro consumado',
 'estupro tentado',
 'estupro de vulnerável consumado',
 'outros c/c/ dignidade sexual']

In [19]:
def get_tables(df:str,choose:str):
    ind = {'capital':1,'demacro':2,'interior':3,'total':4}

    i = ind[df]
    df1 = []
    
    for table in tables[:67]:
        row_data = []
        rows = table.css('tr')[1:]
        for row in rows:
            cell = row.css("td")[i]
            row_data.append(cell.text().strip())
        row_data = np.array(row_data).T
        df1.append(row_data)
        
# Create a DataFrame from the extracted data
    df_df1 = pd.DataFrame(df1, columns = col1).astype('uint8')
    
    df2 = []
    
    for table in tables[67:]:
        row_data = []
        rows = table.css('tr')[1:]
        for row in rows:
            cell = row.css("td")[i]
            row_data.append(cell.text().strip())
        row_data = np.array(row_data).T
        df2.append(row_data)
        
# Create a DataFrame from the extracted data
    df_df2 = pd.DataFrame(df2, columns = col2).astype('uint8')
    
    df = pd.concat([df_df1[col2],df_df2],axis=0, ignore_index=True)
    dfraw = pd.concat([df_df1,df_df2],axis=0, ignore_index=True)
    typee = {'new':df1, 'old':df2,'total':df,'raw':dfraw}
    returned = typee[choose]
    return returned

In [20]:
def create_df_type(df:pd.DataFrame,name:str):
    df = df.assign(regiao=name)
    return df

In [21]:
df_interior = get_tables('interior','total')
df_capital = get_tables('capital','total')
df_demacro = get_tables('demacro','total')
df_total = get_tables('total','total')

In [42]:
df_tabular = pd.concat([pd.concat([create_df_type(get_tables('capital','raw'),'capital'),ocorrencia_time['date']],axis=1),
                        pd.concat([create_df_type(get_tables('demacro','raw'),'demacro'),ocorrencia_time['date']],axis=1),
                        pd.concat([create_df_type(get_tables('interior','raw'),'interior'),ocorrencia_time['date']],axis=1)],
                       axis=0)

In [53]:
df_tabular = df_tabular.sort_values('date').reset_index(drop=True)

In [46]:
df_total_total = create_df_type(get_tables('total','raw'),'total')
df_total_total['date'] = ocorrencia_time['date']

In [56]:
df_tabular['feminicídio'].fillna(0, inplace=True)

In [57]:
df_tabular.to_csv('df_tabular.csv')

In [39]:
# Create a SQLite database and connect to it
conn = sqlite3.connect('ssp_db.db')

#Insert the tables in the db:

df_total.to_sql('crimes_total', conn, if_exists='replace', index=False)
df_interior.to_sql('crimes_interior', conn, if_exists='replace', index=False)
df_demacro.to_sql('crimes_demacro', conn, if_exists='replace', index=False)
df_capital.to_sql('crimes_capital', conn, if_exists='replace', index=False)

# Commit changes
conn.commit()
conn.close()